# Horse Names Analysis: Missing Horses from Herds File

This notebook analyzes which horse names appear in the merged manifest but are missing from the horse_herds file.
We'll implement thorough matching logic to catch variations, abbreviations, and numbering differences.

In [3]:
import pandas as pd
import numpy as np
import re
from collections import defaultdict
import os
from difflib import SequenceMatcher

# Set display options for better readability
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

## 1. Load Data Files

In [4]:
# Define file paths
data_root = os.path.expanduser("~/google-drive/horseID Project/data")
manifest_file = os.path.join(data_root, "horse_photos_manifest_merged.csv")
herds_file = os.path.join(data_root, "horse_herds.csv")

print(f"Loading manifest from: {manifest_file}")
print(f"Loading herds from: {herds_file}")

# Load the data
manifest_df = pd.read_csv(manifest_file)
herds_df = pd.read_csv(herds_file)

print(f"\nManifest shape: {manifest_df.shape}")
print(f"Herds shape: {herds_df.shape}")

# Check the columns
print(f"\nManifest columns: {list(manifest_df.columns)}")
print(f"Herds columns: {list(herds_df.columns)}")

Loading manifest from: /Users/dleigh/google-drive/horseID Project/data/horse_photos_manifest_merged.csv
Loading herds from: /Users/dleigh/google-drive/horseID Project/data/horse_herds.csv

Manifest shape: (5963, 17)
Herds shape: (206, 3)

Manifest columns: ['canonical_id', 'original_canonical_id', 'horse_name', 'email_date', 'message_id', 'original_filename', 'filename', 'date_added', 'num_horses_detected', 'last_merged_timestamp', 'status', 'size_ratio', 'bbox_x', 'bbox_y', 'bbox_width', 'bbox_height', 'segmentation_mask']
Herds columns: ['horse_name', 'herd', 'basename']


In [7]:
herds_df[['basename']].drop_duplicates().sort_values(by='basename')

,basename
125,Absinthe
6,Ace
81,Agave
40,Al
178,Angie
201,Anthem
195,Aries
171,Arvid
176,Atlantis
124,Audi


## 2. Extract and Examine Horse Names

In [3]:
# Extract unique horse names from both datasets
manifest_horses = set(manifest_df['horse_name'].dropna().unique())
herds_horses = set(herds_df['horse_name'].dropna().unique())

print(f"Unique horses in manifest: {len(manifest_horses)}")
print(f"Unique horses in herds: {len(herds_horses)}")

# Show some examples
print(f"\nFirst 10 manifest horses: {sorted(list(manifest_horses))[:10]}")
print(f"First 10 herds horses: {sorted(list(herds_horses))[:10]}")

Unique horses in manifest: 242
Unique horses in herds: 201

First 10 manifest horses: ['Absinthe', 'Ace', 'Agave', 'Al', 'Angie', 'Anthem', 'Aries', 'Artie', 'Arvid', 'Atlantis']
First 10 herds horses: ['Absinthe', 'Ace', 'Agave', 'Al', 'Angie', 'Anthem', 'Aries', 'Arvid', 'Atlantis', 'Audi']


## 3. Test Cases: Verify Known Issues

In [4]:
# Let's check the specific cases mentioned: Cowboy and CeeLo
test_cases = ['Cowboy', 'CeeLo', 'Cee']

print("Testing known cases:")
for case in test_cases:
    in_manifest = case in manifest_horses
    in_herds = case in herds_horses
    print(f"  {case}: Manifest={in_manifest}, Herds={in_herds}")

# Look for Cowboy variations
print("\nCowboy-related names in manifest:")
cowboy_manifest = [name for name in manifest_horses if 'cowboy' in name.lower()]
print(f"  {cowboy_manifest}")

print("\nCowboy-related names in herds:")
cowboy_herds = [name for name in herds_horses if 'cowboy' in name.lower()]
print(f"  {cowboy_herds}")

# Look for Cee variations
print("\nCee-related names in manifest:")
cee_manifest = [name for name in manifest_horses if 'cee' in name.lower()]
print(f"  {cee_manifest}")

print("\nCee-related names in herds:")
cee_herds = [name for name in herds_horses if 'cee' in name.lower()]
print(f"  {cee_herds}")

Testing known cases:
  Cowboy: Manifest=True, Herds=False
  CeeLo: Manifest=True, Herds=False
  Cee: Manifest=False, Herds=False

Cowboy-related names in manifest:
  ['Cowboy']

Cowboy-related names in herds:
  ['Cowboy 2', 'Cowboy 1']

Cee-related names in manifest:
  ['CeeLo']

Cee-related names in herds:
  ['Ceelo']


## 4. Define Name Normalization Functions

In [5]:
def normalize_horse_name(name):
    """
    Normalize a horse name by:
    1. Converting to lowercase
    2. Removing trailing numbers (e.g., "Cowboy 1" -> "cowboy")
    3. Removing extra whitespace
    """
    if pd.isna(name):
        return None
    
    # Convert to lowercase and strip whitespace
    normalized = str(name).lower().strip()
    
    # Remove trailing numbers (e.g., " 1", " 2", etc.)
    normalized = re.sub(r'\s+\d+$', '', normalized)
    
    return normalized

def get_base_name_variations(name):
    """
    Get various possible base name variations for matching.
    Returns a set of possible variations.
    """
    if pd.isna(name):
        return set()
    
    variations = set()
    normalized = normalize_horse_name(name)
    variations.add(normalized)
    
    # Add original case-insensitive version
    variations.add(str(name).lower().strip())
    
    return variations

# Test the normalization
test_names = ['Cowboy 1', 'CeeLo', 'Cee', 'George 2', 'Da Vinci']
print("Testing normalization:")
for name in test_names:
    normalized = normalize_horse_name(name)
    variations = get_base_name_variations(name)
    print(f"  '{name}' -> normalized: '{normalized}', variations: {variations}")

Testing normalization:
  'Cowboy 1' -> normalized: 'cowboy', variations: {'cowboy 1', 'cowboy'}
  'CeeLo' -> normalized: 'ceelo', variations: {'ceelo'}
  'Cee' -> normalized: 'cee', variations: {'cee'}
  'George 2' -> normalized: 'george', variations: {'george 2', 'george'}
  'Da Vinci' -> normalized: 'da vinci', variations: {'da vinci'}


## 5. Create Normalized Name Sets

In [6]:
# Create normalized versions of all names
manifest_normalized = {normalize_horse_name(name): name for name in manifest_horses if normalize_horse_name(name)}
herds_normalized = {normalize_horse_name(name): name for name in herds_horses if normalize_horse_name(name)}

print(f"Normalized manifest horses: {len(manifest_normalized)}")
print(f"Normalized herds horses: {len(herds_normalized)}")

# Create reverse mappings (normalized -> original)
manifest_norm_to_orig = {normalize_horse_name(name): name for name in manifest_horses}
herds_norm_to_orig = {normalize_horse_name(name): name for name in herds_horses}

# Test our known cases
print("\nTesting known cases with normalization:")
test_cases = ['Cowboy', 'CeeLo']
for case in test_cases:
    norm_case = normalize_horse_name(case)
    print(f"  {case} -> {norm_case}")
    print(f"    In manifest normalized: {norm_case in manifest_normalized}")
    print(f"    In herds normalized: {norm_case in herds_normalized}")
    if norm_case in herds_normalized:
        print(f"    Matches herds original: {herds_normalized[norm_case]}")

Normalized manifest horses: 239
Normalized herds horses: 194

Testing known cases with normalization:
  Cowboy -> cowboy
    In manifest normalized: True
    In herds normalized: True
    Matches herds original: Cowboy 1
  CeeLo -> ceelo
    In manifest normalized: True
    In herds normalized: True
    Matches herds original: Ceelo


## 6. Find Exact Matches and Missing Horses

In [7]:
# Find exact matches (after normalization)
matched_normalized = set(manifest_normalized.keys()) & set(herds_normalized.keys())
missing_normalized = set(manifest_normalized.keys()) - set(herds_normalized.keys())

print(f"Horses matched (after normalization): {len(matched_normalized)}")
print(f"Horses missing (after normalization): {len(missing_normalized)}")

# Convert back to original names for reporting
matched_original = [(manifest_normalized[norm], herds_normalized[norm]) for norm in matched_normalized]
missing_original = [manifest_normalized[norm] for norm in missing_normalized]

print(f"\nFirst 10 matched pairs (manifest -> herds):")
for i, (manifest_name, herds_name) in enumerate(sorted(matched_original)[:10]):
    print(f"  {i+1}. '{manifest_name}' -> '{herds_name}'")

print(f"\nFirst 10 missing horses:")
for i, name in enumerate(sorted(missing_original)[:10]):
    print(f"  {i+1}. '{name}'")

Horses matched (after normalization): 149
Horses missing (after normalization): 90

First 10 matched pairs (manifest -> herds):
  1. 'Absinthe' -> 'Absinthe'
  2. 'Ace' -> 'Ace'
  3. 'Agave' -> 'Agave'
  4. 'Al' -> 'Al'
  5. 'Angie' -> 'Angie'
  6. 'Anthem' -> 'Anthem'
  7. 'Aries' -> 'Aries'
  8. 'Arvid' -> 'Arvid'
  9. 'Atlantis' -> 'Atlantis'
  10. 'Audi' -> 'Audi'

First 10 missing horses:
  1. 'Artie'
  2. 'Azar'
  3. 'Baby and Scotty'
  4. 'Bam'
  5. 'Barney'
  6. 'Bear'
  7. 'Benji'
  8. 'Berrypatriot'
  9. 'Booger'
  10. 'Boost'


## 7. Verify Known Problem Cases

In [8]:
# Check if Cowboy and CeeLo are correctly handled
problem_cases = ['Cowboy', 'CeeLo']

print("Verifying problem cases:")
for case in problem_cases:
    if case in missing_original:
        print(f"  ❌ {case} is incorrectly listed as missing")
        # Look for what it should match
        norm_case = normalize_horse_name(case)
        if norm_case in herds_normalized:
            print(f"    Should match: {herds_normalized[norm_case]}")
    else:
        print(f"  ✅ {case} is correctly matched or not in manifest")

# Let's also check what matches we found for these
print("\nActual matches found:")
for manifest_name, herds_name in matched_original:
    if any(case.lower() in manifest_name.lower() for case in problem_cases):
        print(f"  '{manifest_name}' -> '{herds_name}'")

Verifying problem cases:
  ✅ Cowboy is correctly matched or not in manifest
  ✅ CeeLo is correctly matched or not in manifest

Actual matches found:
  'CeeLo' -> 'Ceelo'
  'Cowboy' -> 'Cowboy 1'


## 8. Advanced Fuzzy Matching for Edge Cases

In [9]:
def similarity_score(a, b):
    """Calculate similarity score between two strings."""
    return SequenceMatcher(None, a.lower(), b.lower()).ratio()

def find_fuzzy_matches(missing_horses, herds_set, threshold=0.8):
    """Find potential fuzzy matches for missing horses."""
    fuzzy_matches = {}
    
    for missing in missing_horses:
        best_match = None
        best_score = 0
        
        missing_norm = normalize_horse_name(missing)
        
        for herds_horse in herds_set:
            herds_norm = normalize_horse_name(herds_horse)
            score = similarity_score(missing_norm, herds_norm)
            
            if score > best_score and score >= threshold:
                best_score = score
                best_match = herds_horse
        
        if best_match:
            fuzzy_matches[missing] = (best_match, best_score)
    
    return fuzzy_matches

# Find fuzzy matches for missing horses
fuzzy_matches = find_fuzzy_matches(missing_original, herds_horses, threshold=0.7)

print(f"Found {len(fuzzy_matches)} potential fuzzy matches:")
for missing, (match, score) in sorted(fuzzy_matches.items(), key=lambda x: x[1][1], reverse=True):
    print(f"  '{missing}' -> '{match}' (score: {score:.3f})")

Found 37 potential fuzzy matches:
  'Goodwill' -> 'Good Will' (score: 0.941)
  'Da Vinci' -> 'DaVinci' (score: 0.933)
  'OHalon' -> 'O'Halon' (score: 0.923)
  'Emmet' -> 'Emmett' (score: 0.909)
  'Marky' -> 'Mark' (score: 0.889)
  'Rocky' -> 'Rock' (score: 0.889)
  'Azar' -> 'Zar' (score: 0.857)
  'OHallon' -> 'O'Halon' (score: 0.857)
  'Frey' -> 'Fry' (score: 0.857)
  'Lego' -> 'Leo' (score: 0.857)
  'Rafikki' -> 'Raffiki' (score: 0.857)
  'Conola' -> 'Canola' (score: 0.833)
  'Artie' -> 'Martien' (score: 0.833)
  'Lille' -> 'Lilli' (score: 0.800)
  'Ellie' -> 'Belle' (score: 0.800)
  'Isaac' -> 'Issac' (score: 0.800)
  'Polly' -> 'Solly' (score: 0.800)
  'Robyn' -> 'Robin' (score: 0.800)
  'The Captain' -> 'Captain' (score: 0.778)
  'The Colonel' -> 'Colonel' (score: 0.778)
  'Makia' -> 'Mia' (score: 0.750)
  'Benji' -> 'Ben' (score: 0.750)
  'Sly' -> 'Solly' (score: 0.750)
  'Elroy' -> 'Roy 2' (score: 0.750)
  'Ted' -> 'Teddy' (score: 0.750)
  'Sock' -> 'Rock' (score: 0.750)
  'Berr

## 9. Apply Fuzzy Matches and Update Missing List

In [10]:
# Update our matched and missing lists with fuzzy matches
additional_matches = []
confirmed_missing = []

for horse in missing_original:
    if horse in fuzzy_matches:
        match, score = fuzzy_matches[horse]
        additional_matches.append((horse, match, score))
        print(f"Fuzzy match: '{horse}' -> '{match}' (score: {score:.3f})")
    else:
        confirmed_missing.append(horse)

print(f"\nSummary:")
print(f"  Exact matches: {len(matched_original)}")
print(f"  Additional fuzzy matches: {len(additional_matches)}")
print(f"  Total matches: {len(matched_original) + len(additional_matches)}")
print(f"  Confirmed missing: {len(confirmed_missing)}")
print(f"  Total horses in manifest: {len(manifest_horses)}")

Fuzzy match: 'Lille' -> 'Lilli' (score: 0.800)
Fuzzy match: 'Ellie' -> 'Belle' (score: 0.800)
Fuzzy match: 'Ollie' -> 'Oliver' (score: 0.727)
Fuzzy match: 'Makia' -> 'Mia' (score: 0.750)
Fuzzy match: 'Goodwill' -> 'Good Will' (score: 0.941)
Fuzzy match: 'Markham' -> 'Mark' (score: 0.727)
Fuzzy match: 'Takoda' -> 'Kodak' (score: 0.727)
Fuzzy match: 'Azar' -> 'Zar' (score: 0.857)
Fuzzy match: 'Benji' -> 'Ben' (score: 0.750)
Fuzzy match: 'Isaac' -> 'Issac' (score: 0.800)
Fuzzy match: 'Marky' -> 'Mark' (score: 0.889)
Fuzzy match: 'Sly' -> 'Solly' (score: 0.750)
Fuzzy match: 'Berrypatriot' -> 'Patriot' (score: 0.737)
Fuzzy match: 'Conola' -> 'Canola' (score: 0.833)
Fuzzy match: 'Rocky' -> 'Rock' (score: 0.889)
Fuzzy match: 'Endermann' -> 'Ender' (score: 0.714)
Fuzzy match: 'OHallon' -> 'O'Halon' (score: 0.857)
Fuzzy match: 'Frey' -> 'Fry' (score: 0.857)
Fuzzy match: 'Lego' -> 'Leo' (score: 0.857)
Fuzzy match: 'Dante' -> 'Anthem' (score: 0.727)
Fuzzy match: 'Polly' -> 'Solly' (score: 0.800)


## 10. Categorize Truly Missing Horses

In [11]:
# Categorize the confirmed missing horses
categories = {
    'multi_horse': [],
    'forwarded': [],
    'title_based': [],
    'variations': [],
    'regular': []
}

for horse in confirmed_missing:
    horse_lower = horse.lower()
    
    if ' and ' in horse_lower:
        categories['multi_horse'].append(horse)
    elif horse.startswith('Fwd '):
        categories['forwarded'].append(horse)
    elif horse.startswith('The '):
        categories['title_based'].append(horse)
    elif any(variation in confirmed_missing for variation in ['Da Vinci', 'Davinci', 'DaVinci'] if variation != horse):
        categories['variations'].append(horse)
    else:
        # Check for other name variations
        is_variation = False
        for other in confirmed_missing:
            if other != horse and similarity_score(horse, other) > 0.8:
                categories['variations'].append(horse)
                is_variation = True
                break
        
        if not is_variation:
            categories['regular'].append(horse)

# Print categorization results
print("Categorization of truly missing horses:")
for category, horses in categories.items():
    print(f"\n{category.upper().replace('_', ' ')} ({len(horses)} horses):")
    for horse in sorted(horses):
        print(f"  - {horse}")

Categorization of truly missing horses:

MULTI HORSE (2 horses):
  - Baby and Scotty
  - Cherokee and Custer

FORWARDED (3 horses):
  - Fwd Ace
  - Fwd Tig
  - Fwd Yorick

TITLE BASED (2 horses):
  - The Dude
  - The General

VARIATIONS (4 horses):
  - Fairwind
  - Fairwinds
  - Fritz
  - Fritzy

REGULAR (42 horses):
  - Bam
  - Barney
  - Bear
  - Booger
  - Boost
  - Bourbon
  - Cedric
  - Chance
  - Connor
  - Courage
  - Dizzy
  - Hammer
  - Harry
  - Hermes
  - Jax
  - Kleegal
  - Kohe
  - Kully
  - Laurel
  - Louisdor
  - Macallen
  - Mags
  - Maya
  - Midnight
  - Mike
  - Monster
  - Moon
  - Nathalia
  - Oscar
  - Poncho
  - Quatrero
  - Rolo
  - Rylee
  - Sam
  - Seamus
  - Shalimar
  - Skyman
  - Socks
  - Sugar
  - Swag
  - Versailles
  - Yorick


## 11. Generate Final Report

In [12]:
# Create a comprehensive final report
report_lines = []
report_lines.append("CORRECTED HORSE NAMES ANALYSIS REPORT")
report_lines.append("=====================================")
report_lines.append("")
report_lines.append(f"Analysis Date: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}")
report_lines.append("")

report_lines.append("SUMMARY STATISTICS:")
report_lines.append("-" * 20)
report_lines.append(f"Total horses in merged manifest: {len(manifest_horses)}")
report_lines.append(f"Total horses in herds file: {len(herds_horses)}")
report_lines.append(f"Exact matches (after normalization): {len(matched_original)}")
report_lines.append(f"Additional fuzzy matches: {len(additional_matches)}")
report_lines.append(f"Total horses matched: {len(matched_original) + len(additional_matches)}")
report_lines.append(f"Horses truly missing from herds: {len(confirmed_missing)}")
report_lines.append(f"Match rate: {((len(matched_original) + len(additional_matches)) / len(manifest_horses) * 100):.1f}%")
report_lines.append("")

if additional_matches:
    report_lines.append("FUZZY MATCHES FOUND:")
    report_lines.append("-" * 20)
    for manifest_name, herds_name, score in sorted(additional_matches, key=lambda x: x[2], reverse=True):
        report_lines.append(f"'{manifest_name}' -> '{herds_name}' (similarity: {score:.3f})")
    report_lines.append("")

report_lines.append("MISSING HORSES BY CATEGORY:")
report_lines.append("-" * 30)

category_descriptions = {
    'multi_horse': 'Multi-horse entries (containing "and")',
    'forwarded': 'Forwarded email entries (starting with "Fwd")',
    'title_based': 'Title-based names (starting with "The")',
    'variations': 'Name variations/spelling differences',
    'regular': 'Regular single horse names'
}

for category, horses in categories.items():
    if horses:
        report_lines.append(f"\n{category_descriptions[category].upper()} ({len(horses)} horses):")
        for horse in sorted(horses):
            report_lines.append(f"  - {horse}")

report_lines.append("")
report_lines.append("RECOMMENDATIONS:")
report_lines.append("-" * 15)
report_lines.append("1. Review fuzzy matches above - these may be legitimate matches")
report_lines.append("2. Add the truly missing horses to the master horse location file")
report_lines.append("3. Standardize name variations to prevent future mismatches")
report_lines.append("4. Consider splitting multi-horse entries into separate records")
report_lines.append("5. Clean up forwarded email prefixes in the source data")

# Save the report
report_content = "\n".join(report_lines)
report_file = "/Users/dleigh/git/dlleigh/horse-id/corrected_missing_horses_report.txt"

with open(report_file, 'w') as f:
    f.write(report_content)

print(f"Report saved to: {report_file}")
print("\nFinal Report Preview:")
print("=" * 50)
print(report_content[:1000] + "..." if len(report_content) > 1000 else report_content)

Report saved to: /Users/dleigh/git/dlleigh/horse-id/corrected_missing_horses_report.txt

Final Report Preview:
CORRECTED HORSE NAMES ANALYSIS REPORT

Analysis Date: 2025-07-13 21:37:00

SUMMARY STATISTICS:
--------------------
Total horses in merged manifest: 242
Total horses in herds file: 201
Exact matches (after normalization): 149
Additional fuzzy matches: 37
Total horses matched: 186
Horses truly missing from herds: 53
Match rate: 76.9%

FUZZY MATCHES FOUND:
--------------------
'Goodwill' -> 'Good Will' (similarity: 0.941)
'Da Vinci' -> 'DaVinci' (similarity: 0.933)
'OHalon' -> 'O'Halon' (similarity: 0.923)
'Emmet' -> 'Emmett' (similarity: 0.909)
'Marky' -> 'Mark' (similarity: 0.889)
'Rocky' -> 'Rock' (similarity: 0.889)
'Azar' -> 'Zar' (similarity: 0.857)
'OHallon' -> 'O'Halon' (similarity: 0.857)
'Frey' -> 'Fry' (similarity: 0.857)
'Lego' -> 'Leo' (similarity: 0.857)
'Rafikki' -> 'Raffiki' (similarity: 0.857)
'Conola' -> 'Canola' (similarity: 0.833)
'Artie' -> 'Martien' (simila

## 12. Verification: Double-check Problem Cases

In [13]:
# Final verification of our known problem cases
print("FINAL VERIFICATION OF KNOWN PROBLEM CASES:")
print("=" * 50)

problem_cases = ['Cowboy', 'CeeLo']

for case in problem_cases:
    print(f"\nCase: {case}")
    
    # Check if it's in confirmed missing
    in_missing = case in confirmed_missing
    print(f"  Listed as missing: {in_missing}")
    
    # Check if it's in matched
    matched_with = None
    for manifest_name, herds_name in matched_original:
        if manifest_name == case:
            matched_with = herds_name
            break
    
    if matched_with:
        print(f"  Matched with: {matched_with}")
    
    # Check if it's in fuzzy matches
    fuzzy_match = None
    for manifest_name, herds_name, score in additional_matches:
        if manifest_name == case:
            fuzzy_match = (herds_name, score)
            break
    
    if fuzzy_match:
        print(f"  Fuzzy matched with: {fuzzy_match[0]} (score: {fuzzy_match[1]:.3f})")
    
    # Final status
    if not in_missing:
        print(f"  ✅ Status: Correctly handled (not in missing list)")
    else:
        print(f"  ❌ Status: Problem - still in missing list")

print(f"\n\nFINAL SUMMARY:")
print(f"Total horses truly missing from herds file: {len(confirmed_missing)}")
print(f"Total horses successfully matched: {len(matched_original) + len(additional_matches)}")

FINAL VERIFICATION OF KNOWN PROBLEM CASES:

Case: Cowboy
  Listed as missing: False
  Matched with: Cowboy 1
  ✅ Status: Correctly handled (not in missing list)

Case: CeeLo
  Listed as missing: False
  Matched with: Ceelo
  ✅ Status: Correctly handled (not in missing list)


FINAL SUMMARY:
Total horses truly missing from herds file: 53
Total horses successfully matched: 186
